### **Presenting the Results**

Most of the implementations are in the script file, here we just assemble everything together and present the results for view. 
This is necessary since the script is very long. 
Run the cell below to import everthing from the pre-cooked script file. 



In [1]:
from sqp import *
np.set_printoptions(suppress=True)

The following code uses what is written in the script file and call the SLSQP solver from scipy and solve the problem for a specific strings. 

In [2]:
def run_sqp(test_string, lmbd):
    pbm = ProblemModelingSQP(test_string, lmbd=lmbd)
    n = len(pbm.States)
    # Functional representation of Constraints =================================
    ineq_cons = {'type': 'ineq',
            'fun' : pbm.IneqCon,
            'jac' : pbm.IneqJac
        }
    eq_cons = {'type': 'eq',
            'fun' : pbm.EqnCon,
            'jac' : pbm.EqnJac
        }
    l = pbm.CRowsCount + len(pbm.TransFreqasVec)
    bounds = scipy.optimize.Bounds(
        np.zeros(l) + 1e-10, np.full(l, np.inf)
    )
    # the objective function and gradient ======================================
    objfxn = pbm.ObjFxn
    objgrad = pbm.ObjGrad
    # initial Guess ============================================================
    C = pbm.ConstraintMatrixC
    x0 = pbm.TransitionMatrix.reshape(-1)
    x0 = np.hstack((x0, np.dot(C, x0) + 1)).copy()
    res = scipy.optimize.minimize(
            objfxn, x0, method='SLSQP', jac=objgrad,
            constraints=[eq_cons, ineq_cons], 
            options={'ftol': 1e-14, 'disp': True},
            bounds=bounds
        )
    M = res.x[:n**2].reshape((n, n))
    return M, pbm.TransitionMatrix


The above function takes in a test string consist of characters representing the observed states from a markov chain. 
It will print out the optimize state transition matrix. 
It returns the optimize state transition matrix together with the original state transition matrix estimated via MLE. 
It setup the problem using the code written in the imported script, and then it solves using the `scipy.optimize.minimize` module using sequential quadratic programming. 
Each time the funtion is evaluated, it prints out the objective value of the function and a timestep accurated to miliseconds. 
In the cell below, we test it with some basic input. 

In [3]:
M, StateTransMatrix = run_sqp("AAABACBABBBCCACBCCCBAABC", 0.1)
print("The original MLE estimated transition matrix is: ")
display(StateTransMatrix)
print("The New Transiiton Matrix after the SQP is: ")
display(M)

[2024-02-06 01:07:14.555] Objective Fxn Value = 60.944825513368066
[2024-02-06 01:07:14.555] Objective Fxn Value = 26.968082437584084
[2024-02-06 01:07:14.560] Objective Fxn Value = 25.26808263279905
[2024-02-06 01:07:14.561] Objective Fxn Value = 25.268082652276583
[2024-02-06 01:07:14.562] Objective Fxn Value = 25.268082635316485
[2024-02-06 01:07:14.562] Objective Fxn Value = 25.26808263401498
[2024-02-06 01:07:14.562] Objective Fxn Value = 25.26808263338635
[2024-02-06 01:07:14.562] Objective Fxn Value = 25.26808263308272
[2024-02-06 01:07:14.562] Objective Fxn Value = 25.268082632936068
[2024-02-06 01:07:14.562] Objective Fxn Value = 25.26808263286523
[2024-02-06 01:07:14.562] Objective Fxn Value = 25.268082632831018
[2024-02-06 01:07:14.562] Objective Fxn Value = 25.268082632814494
[2024-02-06 01:07:14.562] Objective Fxn Value = 25.268082632806518
[2024-02-06 01:07:14.562] Objective Fxn Value = 25.268082632802653
[2024-02-06 01:07:14.562] Objective Fxn Value = 25.2680844139916
[2

c:\Users\alto\AppData\Local\anaconda3\envs\operator_lift\lib\site-packages\scipy\optimize\_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


array([[0.42857143, 0.14285714, 0.42857143],
       [0.25      , 0.375     , 0.375     ],
       [0.375     , 0.375     , 0.25      ]])

The New Transiiton Matrix after the SQP is: 


array([[0.33333333, 0.33333334, 0.33333333],
       [0.33333333, 0.33333334, 0.33333333],
       [0.33333333, 0.33333334, 0.33333333]])

We now decrease the value of lamabda, the reguarlizatin term and see how it affects the results of the transition matrix. 

In [6]:
M, StateTransMatrix = run_sqp("AAABACBABBBCCACBCCCBAABC", 0.001)
print("The original MLE estimated transition matrix is: ")
display(StateTransMatrix)
print("The New Transiiton Matrix after the SQP is: ")
display(M)

[2024-02-06 01:08:18.630] Objective Fxn Value = 60.35082578921251
[2024-02-06 01:08:18.631] Objective Fxn Value = 24.37282268775024
[2024-02-06 01:08:18.632] Objective Fxn Value = 24.53695490608988
[2024-02-06 01:08:18.632] Objective Fxn Value = 24.373850741487622
[2024-02-06 01:08:18.632] Objective Fxn Value = 24.372669911914013
[2024-02-06 01:08:18.633] Objective Fxn Value = 24.99529387472261
[2024-02-06 01:08:18.633] Objective Fxn Value = 24.3810478925473
[2024-02-06 01:08:18.633] Objective Fxn Value = 24.37278279073527
[2024-02-06 01:08:18.633] Objective Fxn Value = 24.372685087187147
[2024-02-06 01:08:18.633] Objective Fxn Value = 24.37267318081255
[2024-02-06 01:08:18.633] Objective Fxn Value = 24.37267081109043
[2024-02-06 01:08:18.633] Objective Fxn Value = 24.37267020929039
[2024-02-06 01:08:18.633] Objective Fxn Value = 24.372670025833155
[2024-02-06 01:08:18.634] Objective Fxn Value = 24.37266996038827
[2024-02-06 01:08:18.634] Objective Fxn Value = 24.372669933937612
[2024-

array([[0.42857143, 0.14285714, 0.42857143],
       [0.25      , 0.375     , 0.375     ],
       [0.375     , 0.375     , 0.25      ]])

The New Transiiton Matrix after the SQP is: 


array([[0. , 1. , 0. ],
       [0. , 0.5, 0.5],
       [0.5, 0.5, 0. ]])